# Quantum annealing :

## Test simplifié

Les machines de quantum annealing (comme celles de D-Wave) prennent en entrée un problème sous forme QUBO. Pour cette raison, 
cherchons d'abord à écrire notre hamiltonien sous forme QUBU. \

Selon le résultat établi en [\ref{Equation: particulier}], on a en utilisant les variables binaires:

$H_C=\sum_{i,k}(c_i+\lambda-2\lambda D)x_{i,k}+\lambda \sum_{(i,k)\neq (j,l)} x_{i,k} x_{j,l}+\lambda D^2$

Posons: $a_{i,k}=c_i+2\lambda-2\lambda D$
 et $b_{(i,k),(j,l)}=\lambda$

Soit: $H_{QUBO}=\sum_{i,k}a_{i,k}x_{i,k}+\sum_{(i,k)\neq (j,l)} b_{(i,k),(j,l)}x_{i,k}x_{j,l}+ cte = x^TQx$

## Représentation naive de $g_i$

Dans le code normalement on découpe l'intervalle $[0, P_i]$ en petits pas de taille $\delta$. J'ai mis donc un cas général si on souhaite plus de précision au niveau des résultats. 
 
Si $P_i = 4$ et $\delta = 1$, alors $g_i$ peut prendre les valeurs  
$\{0, 1, 2, 3, 4\}$. Comme fait pour QAOA.

---- Représentation binaire ----


Pour chaque $g_i$, on utilise 
$k_i = \frac{P_i}{\delta}$
variables binaires.  

Chaque variable binaire $x_{i,j}$ représente une portion de $g_i$, et l’on a :
$g_i = \delta \cdot \sum_{j=0}^{k_i - 1} x_{i,j}$.
 
Si $P_i = 4$, $\delta = 1 \Rightarrow k_i = 4$ variables binaires.  
Si $(x_{i,0}, x_{i,1}, x_{i,2}, x_{i,3}) = (1,1,0,0)$, alors :
$g_i = 1 \cdot 1 + 1 \cdot 1 + 0 + 0 = 2$.

## 2 noeuds

In [10]:
import numpy as np
from dwave.samplers import SimulatedAnnealingSampler

# Paramètres du problème
P_i = [4, 2]       # Puissance max par noeud
c_i = [3, 8]        # Coût par unité de puissance
D_i = [3, 1]        # Demande par noeud
delta = 1           # Pas de discrétisation
lam = 1000          # Coefficient de pénalité (augmenté)

# la demande totale D
D = np.sum(D_i)

# Nombre de variables binaires par noeud
k = [int(p / delta) for p in P_i]  #[4,2]
total_vars = sum(k)                 # 6 variables binaires

# Mapping des variables (i, j) -> index global
mapping = {}
index = 0
for i in range(len(P_i)):
    for j in range(k[i]):
        mapping[(i, j)] = index
        index += 1

# Construction du QUBO
Q = {}
for i in range(len(P_i)):
    for j in range(k[i]):
        u = mapping[(i, j)]
        # Terme linéaire: coût + pénalité
        Q[(u, u)] = c_i[i] * delta + lam * (delta**2 - 2 * delta * D)
        
        # Termes quadratiques (paires de variables)
        for v in range(u + 1, total_vars):
            Q[(u, v)] = 2 * lam * (delta**2)

# Résolution avec quantum annealing
sampler = SimulatedAnnealingSampler()
sampleset = sampler.sample_qubo(Q, num_reads=1000)
sample = sampleset.first.sample

#Reconstruction des g_i
g = [0] * len(P_i)
for i in range(len(P_i)):
    for j in range(k[i]):
        if sample[mapping[(i, j)]] == 1:
            g[i] += delta

# Vérification des résultats
print(f"Solution optimale: g = {g}")
print(f"Coût total: {sum(c_i[i] * g[i] for i in range(len(g)))}")
print(f"Demande totale: {sum(g)} (cible: {D})")

Solution optimale: g = [4, 0]
Coût total: 12
Demande totale: 4 (cible: 4)


## Résolution classique

In [54]:
import gurobipy as gp
from gurobipy import GRB


noeuds = [0, 1, 2, 3]
P_i = [15, 20, 10, 25]  # Puissances max (MW)
c_i = [40, 35, 50, 30]  # Coûts (€/MWh)
D_i = [12, 8, 5, 15]    # Demandes (MW)
D_total = sum(D_i)       # Demande totale (40 MW)

model = gp.Model("OptimisationProduction")
model.setParam('OutputFlag', 0) # pour n'afficher que le résultat final (sans détails)

# Variables de décision
g = model.addVars(noeuds, lb=0.0, vtype=GRB.CONTINUOUS, name="production")

#Fonction objectif (minimiser les coûts)
model.setObjective(gp.quicksum(c_i[i] * g[i] for i in noeuds), GRB.MINIMIZE)

#Contraintes de capacité
capacity_constrs = model.addConstrs(
    (g[i] <= P_i[i] for i in noeuds), 
    name="capacity"
)

#Contrainte d'équilibre offre-demande
balance_constr = model.addConstr(
    gp.quicksum(g[i] for i in noeuds) == D_total, 
    name="balance"
)

model.optimize()
if model.status == GRB.OPTIMAL:
    print("\nSOLUTION OPTIMALE:")
    print(f"Coût total: {model.objVal:.2f} €")
    print(f"Demande totale: {D_total} MW")
    
    total_prod = 0
    for i in noeuds:
        prod_val = g[i].X
        total_prod += prod_val
        print(f"Nœud {i+1}: {prod_val:.1f}/{P_i[i]} MW " f"(coût: {c_i[i]} €/MWh)")
    
    print(f"\nProduction totale: {total_prod} MW")
    print("Vérification contraintes:")
    print(f"- Équilibre offre/demande: {'OK' if abs(total_prod - D_total) < 1e-6 else 'ERREUR'}")
    print("- Capacités respectées: ", end="")
    for i in noeuds:
        if g[i].X > P_i[i]:
            print(f"\n  ERREUR nœud {i+1} ({g[i].X} > {P_i[i]})", end="")
    print("OK" if all(g[i].X <= P_i[i] for i in noeuds) else "")
    
    


SOLUTION OPTIMALE:
Coût total: 1275.00 €
Demande totale: 40 MW
Nœud 1: 0.0/15 MW (coût: 40 €/MWh)
Nœud 2: 15.0/20 MW (coût: 35 €/MWh)
Nœud 3: 0.0/10 MW (coût: 50 €/MWh)
Nœud 4: 25.0/25 MW (coût: 30 €/MWh)

Production totale: 40.0 MW
Vérification contraintes:
- Équilibre offre/demande: OK
- Capacités respectées: OK


Cela montre bien qu'on trouve la solution optimale. 

# Utilisation du développement binaire

Comme il y a le risque de dépasser la valeur de $P_i$ on rajoute une contrainte d'inégalité

### 2 noeuds

In [148]:
import numpy as np
import math
from dwave.samplers import SimulatedAnnealingSampler



# Paramètres du problème
P_i = [4, 2]       # Puissance max par noeud
c_i = [3, 8]        # Coût par unité de puissance
D_i = [3, 1]        # Demande par noeud
D_total = sum(D_i)       # 40 MW

# Coefficients de pénalité RENFORCÉS
lam= 10 


n_bits = [math.ceil(math.log2(p + 1)) for p in P_i]  # [4, 5, 4, 5]
total_vars = sum(n_bits)  

print(f"Nombre total de variables binaires: {total_vars}")

# Mapping des variables (i, bit) → index global
mapping = {}
index = 0
for i in range(len(P_i)):
    for j in range(n_bits[i]):
        mapping[(i, j)] = index
        index += 1

# CONSTRUCTION DU QUBO OPTIMISÉ
Q = {}

# 1. Termes de coût (linéaires)
for i in range(len(P_i)):
    for j in range(n_bits[i]):
        idx = mapping[(i, j)]
        weight = 2**j
        Q[(idx, idx)] = c_i[i] * weight

# Contrainte d'égalité (∑g_i = D_total) 
for idx1 in range(total_vars):
    for idx2 in range(idx1, total_vars):
        # Trouver les poids binaires
        a = next(2**j for (i,j), idx in mapping.items() if idx == idx1)
        b = a if idx1 == idx2 else next(2**j for (i,j), idx in mapping.items() if idx == idx2)
        
        coef = lam * a * b
        if idx1 == idx2:
            Q[(idx1, idx1)] = Q.get((idx1, idx1), 0) + coef - 2 * lam * D_total * a
        else:
            Q[(idx1, idx2)] = Q.get((idx1, idx2), 0) + 2 * coef

# Ajout du terme constant (nécessaire pour l'égalité)
constant_term = lam* D_total**2


sampler = SimulatedAnnealingSampler()
sampleset = sampler.sample_qubo(Q, num_reads=1000, seed=1234)  # Seed pour reproductibilité
sample = sampleset.first.sample



g = [0] * len(P_i)
for i in range(len(P_i)):
    for j in range(n_bits[i]):
        if sample[mapping[(i, j)]] == 1:
            g[i] += 2**j


total_production = sum(g)
total_cost = sum(c_i[i] * g[i] for i in range(len(g)))

print("\n RÉSULTATS QA:")
print(f"Production: {g} MW")
print(f"Production totale: {total_production} MW")
print(f"Demande totale: {D_total} MW")
print(f"Coût total: {total_cost} €")
print(f"Valeur du terme constant: {constant_term}")

print("\nContraintes de capacité:")
for i, (prod, p_max) in enumerate(zip(g, P_i)):
    status = "OK" if prod <= p_max else f"DÉPASSEMENT ({prod} > {p_max})"
    print(f"Nœud {i+1}: {prod}/{p_max} MW - {status}")

print("\nÉquilibre offre/demande:", "OK" if total_production == D_total else f"ERREUR ({total_production} ≠ {D_total})")

Nombre total de variables binaires: 5

 RÉSULTATS QA:
Production: [4, 0] MW
Production totale: 4 MW
Demande totale: 4 MW
Coût total: 12 €
Valeur du terme constant: 160

Contraintes de capacité:
Nœud 1: 4/4 MW - OK
Nœud 2: 0/2 MW - OK

Équilibre offre/demande: OK


On peut avoir accès à la matrice Q construite et vérifier s'il est bien construite comme suit:

In [147]:
import numpy as np
import math
from dwave.samplers import SimulatedAnnealingSampler

# Paramètres du problème
P_i = [4, 2]       # Puissance max par noeud
c_i = [3, 8]        # Coût par unité de puissance
D_i = [3, 1]        # Demande par noeud
D_total = sum(D_i)       # 40 MW

# Coefficients de pénalité RENFORCÉS
lam = 10 

n_bits = [math.ceil(math.log2(p + 1)) for p in P_i]  # [4, 5, 4, 5]
total_vars = sum(n_bits)  

print(f"Nombre total de variables binaires (qubits): {total_vars}")
print(f"Nombre de bits par nœud: {n_bits}")

# Mapping des variables (i, bit) → index global
mapping = {}
index = 0
for i in range(len(P_i)):
    for j in range(n_bits[i]):
        mapping[(i, j)] = index
        index += 1

# CONSTRUCTION DU QUBO OPTIMISÉ
Q = {}

# 1. Termes de coût (linéaires)
for i in range(len(P_i)):
    for j in range(n_bits[i]):
        idx = mapping[(i, j)]
        weight = 2**j
        Q[(idx, idx)] = c_i[i] * weight

# Contrainte d'égalité (∑g_i = D_total) 
for idx1 in range(total_vars):
    for idx2 in range(idx1, total_vars):
        # Trouver les poids binaires
        a = next(2**j for (i,j), idx in mapping.items() if idx == idx1)
        b = a if idx1 == idx2 else next(2**j for (i,j), idx in mapping.items() if idx == idx2)
        
        coef = lam * a * b
        if idx1 == idx2:
            Q[(idx1, idx1)] = Q.get((idx1, idx1), 0) + coef - 2 * lam * D_total * a
        else:
            Q[(idx1, idx2)] = Q.get((idx1, idx2), 0) + 2 * coef

# Ajout du terme constant (nécessaire pour l'égalité)
constant_term = lam * D_total**2

# AFFICHAGE DE LA MATRICE Q
print("\n" + "="*50)
print("MATRICE Q CONSTRUITE:")
print("="*50)

# Créer une matrice numpy pour une visualisation plus claire
Q_matrix = np.zeros((total_vars, total_vars))
for (i, j), value in Q.items():
    Q_matrix[i, j] = value
    if i != j:  # La matrice est symétrique
        Q_matrix[j, i] = value

print("Matrice Q complète:")
print(Q_matrix)

print(f"\nNombre d'éléments non nuls dans Q: {np.count_nonzero(Q_matrix)}")
print(f"Densité de la matrice: {np.count_nonzero(Q_matrix) / (total_vars**2) * 100:.2f}%")

# Afficher les termes diagonaux (linéaires)
print("\nTermes diagonaux (linéaires):")
for i in range(total_vars):
    print(f"Q[{i},{i}] = {Q_matrix[i,i]:.2f}")

# Afficher quelques termes non diagonaux (quadratiques)
print("\nQuelques termes non diagonaux (quadratiques):")
count = 0
for i in range(total_vars):
    for j in range(i+1, total_vars):
        if Q_matrix[i,j] != 0 and count < 5:  # Afficher seulement les 5 premiers non nuls
            print(f"Q[{i},{j}] = {Q_matrix[i,j]:.2f}")
            count += 1



Nombre total de variables binaires (qubits): 5
Nombre de bits par nœud: [3, 2]

MATRICE Q CONSTRUITE:
Matrice Q complète:
[[ -67.   40.   80.   20.   40.]
 [  40. -114.  160.   40.   80.]
 [  80.  160. -148.   80.  160.]
 [  20.   40.   80.  -62.   40.]
 [  40.   80.  160.   40. -104.]]

Nombre d'éléments non nuls dans Q: 25
Densité de la matrice: 100.00%

Termes diagonaux (linéaires):
Q[0,0] = -67.00
Q[1,1] = -114.00
Q[2,2] = -148.00
Q[3,3] = -62.00
Q[4,4] = -104.00

Quelques termes non diagonaux (quadratiques):
Q[0,1] = 40.00
Q[0,2] = 80.00
Q[0,3] = 20.00
Q[0,4] = 40.00
Q[1,2] = 160.00


### 4 noeuds

In [59]:
# Paramètres du problème
P_i = [15, 20, 10, 25]  
c_i = [40, 35, 50, 30]  
D_i = [12, 8, 5, 15] 
D_total = sum(D_i)       

lambda1 = 20000


n_bits = [math.ceil(math.log2(p + 1)) for p in P_i]  
total_vars = sum(n_bits) 

print(f"Nombre total de variables binaires: {total_vars}")

# Mapping des variables (i, bit) → index global
mapping = {}
index = 0
for i in range(len(P_i)):
    for j in range(n_bits[i]):
        mapping[(i, j)] = index
        index += 1

# CONSTRUCTION DU QUBO OPTIMISÉ
Q = {}

#Termes de coût (linéaires)
for i in range(len(P_i)):
    for j in range(n_bits[i]):
        idx = mapping[(i, j)]
        weight = 2**j
        Q[(idx, idx)] = c_i[i] * weight

#Contrainte d'égalité (∑g_i = D_total)
for idx1 in range(total_vars):
    for idx2 in range(idx1, total_vars):
        # Trouver les poids binaires
        a = next(2**j for (i,j), idx in mapping.items() if idx == idx1)
        b = a if idx1 == idx2 else next(2**j for (i,j), idx in mapping.items() if idx == idx2)
        
        coef = lambda1 * a * b
        if idx1 == idx2:
            Q[(idx1, idx1)] = Q.get((idx1, idx1), 0) + coef - 2 * lambda1 * D_total * a
        else:
            Q[(idx1, idx2)] = Q.get((idx1, idx2), 0) + 2 * coef

# Ajout du terme constant (nécessaire pour l'égalité)
constant_term = lambda1 * D_total**2


sampler = SimulatedAnnealingSampler()
sampleset = sampler.sample_qubo(Q, num_reads=1000, seed=1234)  # Seed pour reproductibilité
sample = sampleset.first.sample

g = [0] * len(P_i)
for i in range(len(P_i)):
    for j in range(n_bits[i]):
        if sample[mapping[(i, j)]] == 1:
            g[i] += 2**j

total_production = sum(g)
total_cost = sum(c_i[i] * g[i] for i in range(len(g)))

print("\n RÉSULTATS QA:")
print(f"Production: {g} MW")
print(f"Production totale: {total_production} MW")
print(f"Demande totale: {D_total} MW")
print(f"Coût total: {total_cost} €")
print(f"Valeur du terme constant: {constant_term}")

print("\nContraintes de capacité:")
for i, (prod, p_max) in enumerate(zip(g, P_i)):
    status = "OK" if prod <= p_max else f"DÉPASSEMENT ({prod} > {p_max})"
    print(f"Nœud {i+1}: {prod}/{p_max} MW - {status}")

print("\nÉquilibre offre/demande:", "OK" if total_production == D_total else f"ERREUR ({total_production} ≠ {D_total})")

Nombre total de variables binaires: 18

 RÉSULTATS QA:
Production: [4, 5, 0, 31] MW
Production totale: 40 MW
Demande totale: 40 MW
Coût total: 1265 €
Valeur du terme constant: 32000000

Contraintes de capacité:
Nœud 1: 4/15 MW - OK
Nœud 2: 5/20 MW - OK
Nœud 3: 0/10 MW - OK
Nœud 4: 31/25 MW - DÉPASSEMENT (31 > 25)

Équilibre offre/demande: OK


Avec la représentation binaire on a le risque du dépassement de la valeur maximale de $g_i$, pour cela on rajoute la contrainte d'inégalité dans la formulation QUBO.

In [ ]:

P_i = [15, 20, 10, 25]   # Capacités max
c_i = [40, 35, 50, 30]   # Coûts unitaires
D_i = [12, 8, 5, 15]     
D_total = sum(D_i)       # Demande totale

# Coefficients de pénalité
lambda1 = 20000  # Contrainte d'égalité (∑g_i = D_total)
lambda2 = 1000    # Contrainte de capacité (g_i ≤ P_i)


n_bits = [math.ceil(math.log2(p + 1)) for p in P_i]              # bits pour g_i
slack_bits = [math.ceil(math.log2(P_i[i] + 1)) for i in range(len(P_i))]  # bits slack

total_vars = sum(n_bits) + sum(slack_bits)
print(f"Nombre total de variables binaires: {total_vars}")

# Mapping des indices
mapping = {}
index = 0

# Bits g_i
for i in range(len(P_i)):
    for j in range(n_bits[i]):
        mapping[("g", i, j)] = index
        index += 1

# Bits slack s_i
for i in range(len(P_i)):
    for j in range(slack_bits[i]):
        mapping[("s", i, j)] = index
        index += 1

# Construction du QUBO
Q = {}

# Terme de coût : c_i * g_i
for i in range(len(P_i)):
    for j in range(n_bits[i]):
        idx = mapping[("g", i, j)]
        Q[(idx, idx)] = c_i[i] * (2**j)

# Contrainte d'égalité (∑g_i = D_total)
for i1 in range(len(P_i)):
    for j1 in range(n_bits[i1]):
        idx1 = mapping[("g", i1, j1)]
        a = 2**j1
        
        # Terme linéaire
        Q[(idx1, idx1)] = Q.get((idx1, idx1), 0) + lambda1 * (a**2 - 2 * D_total * a)
        
        # Termes croisés g_i-g_j
        for i2 in range(len(P_i)):
            for j2 in range(n_bits[i2]):
                idx2 = mapping[("g", i2, j2)]
                if idx1 < idx2:
                    b = 2**j2
                    Q[(idx1, idx2)] = Q.get((idx1, idx2), 0) + lambda1 * 2 * a * b

# Contrainte de capacité via slack : g_i + s_i = P_i
for i in range(len(P_i)):
    # Bits g_i
    for j1 in range(n_bits[i]):
        idx1 = mapping[("g", i, j1)]
        a = 2**j1
        Q[(idx1, idx1)] = Q.get((idx1, idx1), 0) + lambda2 * (a**2 - 2 * P_i[i] * a)
        
        # Croisés g_i-g_i
        for j2 in range(j1 + 1, n_bits[i]):
            idx2 = mapping[("g", i, j2)]
            b = 2**j2
            Q[(idx1, idx2)] = Q.get((idx1, idx2), 0) + lambda2 * 2 * a * b
        
        # Croisés g_i - slack
        for k in range(slack_bits[i]):
            idx_s = mapping[("s", i, k)]
            w = 2**k
            Q[(idx1, idx_s)] = Q.get((idx1, idx_s), 0) + lambda2 * 2 * a * w

    # Bits slack
    for j1 in range(slack_bits[i]):
        idx1 = mapping[("s", i, j1)]
        w1 = 2**j1
        Q[(idx1, idx1)] = Q.get((idx1, idx1), 0) + lambda2 * (w1**2 - 2 * P_i[i] * w1)
        
        # Croisés slack-slack
        for j2 in range(j1 + 1, slack_bits[i]):
            idx2 = mapping[("s", i, j2)]
            w2 = 2**j2
            Q[(idx1, idx2)] = Q.get((idx1, idx2), 0) + lambda2 * 2 * w1 * w2

# Résolution
sampler = SimulatedAnnealingSampler()
sampleset = sampler.sample_qubo(Q, num_reads=1000)
sample = sampleset.first.sample

# Reconstruction de g_i
g = [0] * len(P_i)
for i in range(len(P_i)):
    for j in range(n_bits[i]):
        if sample[mapping[("g", i, j)]] == 1:
            g[i] += 2**j

# Reconstruction du slack
s = [0] * len(P_i)
for i in range(len(P_i)):
    for j in range(slack_bits[i]):
        if sample[mapping[("s", i, j)]] == 1:
            s[i] += 2**j

# Affichage des résultats
total_production = sum(g)
total_cost = sum(c_i[i] * g[i] for i in range(len(g)))

print("\nRÉSULTATS QA:")
print(f"Production: {g} MW")
print(f"Slack: {s} MW")
print(f"Production totale: {total_production} MW")
print(f"Demande totale: {D_total} MW")
print(f"Coût total: {total_cost} €")

print("\nContraintes de capacité:")
for i, (prod, p_max) in enumerate(zip(g, P_i)):
    status = "OK" if prod <= p_max else f"DÉPASSEMENT ({prod} > {p_max})"
    print(f"Nœud {i+1}: {prod}/{p_max} MW - {status}")

print("\nÉquilibre offre/demande:", "OK" if total_production == D_total else f"ERREUR ({total_production} ≠ {D_total})")


Nombre total de variables binaires: 36

RÉSULTATS QA:
Production: [0, 15, 0, 25] MW
Slack: [15, 5, 10, 0] MW
Production totale: 40 MW
Demande totale: 40 MW
Coût total: 1275 €

Contraintes de capacité:
Nœud 1: 0/15 MW - OK
Nœud 2: 15/20 MW - OK
Nœud 3: 0/10 MW - OK
Nœud 4: 25/25 MW - OK

Équilibre offre/demande: OK


# Résolution de l'OPF:

## Résolution classique:

In [ ]:
from gurobipy import Model, GRB

# Données
nodes = [0, 1]
P_min = {0: 0, 1: 0}
P_max = {0: 4, 1: 6}
c = {0: 10, 1: 5}
d = {0: 3, 1: 3}

lines = [(0,1)]
L_max = {(0,1): 3}
B = {(0,1): 1.0}  # susceptance
X = {(0,1): 1.0 / B[(0,1)]}  # reactance

# Modèle
m = Model("OTS")
m.setParam('OutputFlag', 0)

# Variables
g = m.addVars(nodes, lb=0, ub=GRB.INFINITY, name="g")
l = m.addVars(lines, lb=-GRB.INFINITY, ub=GRB.INFINITY, name="l")
theta = m.addVars(nodes, lb=-GRB.INFINITY, ub=GRB.INFINITY, name="theta")
# delta = m.addVars(lines, vtype=GRB.BINARY, name="delta")  # Supprimé pour OPF



# Contraintes

for i in nodes:
    m.addConstr(g[i] >= P_min[i], name=f"Pmin_{i}")
    m.addConstr(g[i] <= P_max[i], name=f"Pmax_{i}")

m.addConstr(theta[0] == 0, name="ref_angle")

for i in nodes:
    flux_sum = 0
    for (u,v) in lines:
        if u == i:
            flux_sum += l[(u,v)]
        elif v == i:
            flux_sum -= l[(u,v)]
    m.addConstr(g[i] - d[i] == flux_sum, name=f"balance_{i}")

# Suppression des contraintes sur delta et modification des contraintes sur l:

for (i,j) in lines:
    # On considère la ligne toujours fermée => delta=1 implicite
    m.addConstr(l[(i,j)] <= L_max[(i,j)], name=f"cap_pos_{i}_{j}")
    m.addConstr(l[(i,j)] >= -L_max[(i,j)], name=f"cap_neg_{i}_{j}")

    # Contraintes DC sans delta ni Big M
    m.addConstr(l[(i,j)] == (theta[i] - theta[j]) / X[(i,j)], name=f"dc_{i}_{j}")

# Objectif
m.setObjective(sum(c[i]*g[i] for i in nodes), GRB.MINIMIZE)

# Résolution
m.optimize()

# Affichage
if m.status == GRB.OPTIMAL:
    print("Solution optimale trouvée:")
    for i in nodes:
        print(f"g[{i}] = {g[i].X:.2f}")
    for (i,j) in lines:
        print(f"l[{i},{j}] = {l[(i,j)].X:.2f}")
        print(f"theta[{i}] = {theta[i].X:.2f}")
else:
    print("Pas de solution optimale trouvée.")
    

print(theta)


Solution optimale trouvée:
g[0] = 0.00
g[1] = 6.00
l[0,1] = -3.00
theta[0] = 0.00
{0: <gurobi.Var theta[0] (value 0.0)>, 1: <gurobi.Var theta[1] (value 3.0)>}


## Résolution avec QA

Commençons par un exemple de 2 noeuds:

In [ ]:
import dimod
import numpy as np
import math
from collections import defaultdict
from dwave.samplers import SimulatedAnnealingSampler

# Les paramètres du prb
B = [0, 1]  # Noeuds
L = [(0, 1)]  # Lignes (non orientés)
c_i = {0: 10, 1: 5}          # Coûts par nœud
D_i = {0: 3, 1: 3}           # Demande par noeud
B_ij = {(0,1): 1}              # susceptance
lambda_val = 10000          # Coefficient de pénalité

# Bornes maximales 
P_i_max = {0: 4, 1: 6}     # Valeurs maximales pour P_i
L_ij_max = {(0,1): 3}           # Valeurs maximales pour L_ij

P_i = {0: 0, 1: 6}           # Valeurs cibles P_i
L_ij = {(0,1): -3}               # flux entre deux noeuds



# Calcul du nombre de bits nécessaires
p_i = {i: math.ceil(math.log2(P_i_max[i] + 1)) for i in B}
k_ij = {(i,j): math.ceil(math.log2(L_ij_max[(i,j)] + 1)) for (i,j) in L}
K_ij = {(i,j): B_ij[(i,j)] * np.pi / 180.0 for (i,j) in L}  # Facteur de conversion

# CONSTRUCTION DU QUBO
bqm = dimod.BinaryQuadraticModel.empty(dimod.BINARY)



# Ajout des variables binaires
# Variables β (angles)
beta_vars = {}
for i in B:
    for m in range(9):
        beta_vars[(i, m)] = f'beta_{i}_{m}'
        bqm.add_variable(beta_vars[(i, m)], 0.0)

# Variables de correction pour P_i
tilde_c_vars = {}
for i in B:
    for r in range(int(p_i[i])):
        tilde_c_vars[(i, r)] = f'tilde_c_{i}_{r}'
        bqm.add_variable(tilde_c_vars[(i, r)], 0.0)

# Variables de correction pour L_ij (deux sets par lien)
c1_vars = {}
c2_vars = {}
for (i,j) in L:
    for p in range(int(k_ij[(i,j)])):
        c1_vars[(i,j,p)] = f'c1_{i}_{j}_{p}'
        c2_vars[(i,j,p)] = f'c2_{i}_{j}_{p}'
        bqm.add_variable(c1_vars[(i,j,p)], 0.0)
        bqm.add_variable(c2_vars[(i,j,p)], 0.0)

#Terme de coût principal (linéaire)
for i in B:
    for (i,j) in L:
        if (i,j) in K_ij:
            for m in range(8):
                coeff = c_i[i] * K_ij[(i,j)] * (2**m)
                
                # Terme β^i_m
                bqm.add_linear(beta_vars[(i, m)], coeff)
                
                # Terme β^j_m
                bqm.add_linear(beta_vars[(j, m)], -coeff)

# Contraintes quadratiques (pénalités)
def add_quadratic_penalty(bqm, expression, lambda_val, target):
    """
    Ajoute une pénalité quadratique λ*(expression - target)^2 au BQM.
    
    Args:
        expression: Liste de tuples (nom_variable, coefficient)
        lambda_val: Coefficient de pénalité
        target: Valeur cible
    """
    # Développement de (expression - target)^2 = expression^2 - 2*target*expression + target^2
    # On ignore target^2 (constant)
    
    # Termes linéaires et quadratiques de expression^2
    for (var1, coeff1) in expression:
        # Terme linéaire: coeff1^2 * var1 (car var1^2 = var1)
        bqm.add_linear(var1, lambda_val * coeff1**2)
        
        # Terme croisé: 2*coeff1*coeff2 * var1*var2
        for (var2, coeff2) in expression:
            if var1 < var2:  # Évite les doublons
                bqm.add_quadratic(var1, var2, 2 * lambda_val * coeff1 * coeff2)
    
    # Terme -2*λ*target*expression
    for (var, coeff) in expression:
        bqm.add_linear(var, -2 * lambda_val * target * coeff)

# Contraintes pour P_i (nœuds)
for i in B:
    expr = []
    
    # Partie angulaire
    for (i,j) in L:
        if (i,j) in K_ij:
            for m in range(8):
                expr.append((beta_vars[(i, m)], K_ij[(i,j)] * (2**m)))
                expr.append((beta_vars[(j, m)], -K_ij[(i,j)] * (2**m)))
    
    # Variables de correction
    for r in range(int(p_i[i])):
        expr.append((tilde_c_vars[(i, r)], 2**r))
    
    # Ajout de la constante D_i et de la cible P_i
    add_quadratic_penalty(bqm, expr, lambda_val, P_i[i] - D_i[i])

# Contraintes pour L_ij (liens)
for (i,j) in L:
    if (i,j) in K_ij:
        # Première contrainte (c^2_{p,i,j})
        expr1 = []
        for m in range(8):
            expr1.append((beta_vars[(j, m)], K_ij[(i,j)] * (2**m)))
            expr1.append((beta_vars[(i, m)], -K_ij[(i,j)] * (2**m)))
        for p in range(int(k_ij[(i,j)])):
            expr1.append((c2_vars[(i,j,p)], 2**p))
        add_quadratic_penalty(bqm, expr1, lambda_val, L_ij[(i,j)])
        
        # Deuxième contrainte (c^1_{p,i,j})
        expr2 = []
        for m in range(8):
            expr2.append((beta_vars[(i, m)], K_ij[(i,j)] * (2**m)))
            expr2.append((beta_vars[(j, m)], -K_ij[(i,j)] * (2**m)))
        for p in range(int(k_ij[(i,j)])):
            expr2.append((c1_vars[(i,j,p)], 2**p))
        add_quadratic_penalty(bqm, expr2, lambda_val, L_ij[(i,j)])



#La résolution

"""#Pour le recuit classique: (saut thermique) si on voulait faire la comparaison entre les deux recuits 
sampler = dimod.SimulatedAnnealingSampler()
sampleset = sampler.sample(bqm, num_reads=1000)
best_sample = sampleset.first.sample"""

sampler = SimulatedAnnealingSampler()
sampleset = sampler.sample(bqm, num_reads=1000)
best_sample = sampleset.first.sample


def calculate_angle(beta_vars, i):
    return sum(int(best_sample[beta_vars[(i, m)]]) * (2**m) for m in range(9))


def calculate_correction(vars_dict):
    """Calcule la valeur de correction à partir des bits"""
    return sum(best_sample[var] * (2**idx) for (_, var), idx in zip(vars_dict.items(), range(len(vars_dict))))

print("="*50)
print(f"Énergie de la solution: {sampleset.first.energy}")
print("="*50)



# Angles calculés
for i in B:
    angle = calculate_angle(beta_vars, i)
    print(f"Angle au nœud {i}: {angle} degrés")

# Corrections P_i
for i in B:
    correction = calculate_correction({k: v for k, v in tilde_c_vars.items() if k[0] == i})
    total = sum(
        K_ij.get((i,j), 0) * (calculate_angle(beta_vars, i) - calculate_angle(beta_vars, j))
        for j in B if (i,j) in L or (j,i) in L
    ) + D_i[i] + correction
    print(f"Contrainte P_{i}: {total} (cible: {P_i[i]}, correction: {correction})")

# Corrections L_ij
for (i,j) in L:
    angle_diff = K_ij[(i,j)] * (calculate_angle(beta_vars, i) - calculate_angle(beta_vars, j))
    
    correction1 = calculate_correction({k: v for k, v in c1_vars.items() if k[:2] == (i,j)})
    total1 = angle_diff + correction1
    print(f"Contrainte L1_{i}{j}: {total1} (cible: {L_ij[(i,j)]}, correction: {correction1})")
    
    correction2 = calculate_correction({k: v for k, v in c2_vars.items() if k[:2] == (i,j)})
    total2 = -angle_diff + correction2
    print(f"Contrainte L2_{i}{j}: {total2} (cible: {L_ij[(i,j)]}, correction: {correction2})")
    
    
# Nombre total de qubits (variables binaires dans le BQM)
n_qubits = len(bqm.variables)
print(f"Nombre total de qubits logiques utilisés: {n_qubits}")


Énergie de la solution: -90029.98099925066
Angle au nœud 0: 64 degrés
Angle au nœud 1: 406 degrés
Contrainte P_0: -2.969026041820607 (cible: 0, correction: 0)
Contrainte P_1: 10 (cible: 6, correction: 7)
Contrainte L1_01: -5.969026041820607 (cible: -3, correction: 0)
Contrainte L2_01: 5.969026041820607 (cible: -3, correction: 0)
Nombre total de qubits logiques utilisés: 28


Je voulais vérifier également la construction de BQM:

In [ ]:
def verifier_construction_bqm(bqm, variables_info):
    """Vérifie systématiquement la construction du BQM"""
    
    print("\n" + "="*60)
    print("VÉRIFICATION DE LA CONSTRUCTION DU BQM")
    print("="*60)
    
    # 1. Vérification de la symétrie des termes quadratiques
    est_symetrique = True
    for (var1, var2), coeff in bqm.quadratic.items():
        if (var2, var1) not in bqm.quadratic or abs(bqm.quadratic[(var2, var1)] - coeff) > 1e-10:
            est_symetrique = False
            print(f"  ERREUR: Terme asymétrique ({var1}, {var2}) = {coeff} vs ({var2}, {var1}) = {bqm.quadratic.get((var2, var1), 'absent')}")
    
    print(f"✓ Matrice symétrique: {est_symetrique}")
    
    # 2. Vérification des variables déclarées
    variables_bqm = set(bqm.variables)
    variables_attendues = set()
    
    for var_type, vars_dict in variables_info.items():
        variables_attendues.update(vars_dict.values())
    
    manquantes = variables_attendues - variables_bqm
    supplementaires = variables_bqm - variables_attendues
    
    print(f"✓ Variables manquantes: {len(manquantes)}")
    print(f"✓ Variables supplémentaires: {len(supplementaires)}")
    
    if manquantes:
        print(f"  Variables attendues mais manquantes: {list(manquantes)[:5]}{'...' if len(manquantes) > 5 else ''}")
    if supplementaires:
        print(f"  Variables présentes mais non attendues: {list(supplementaires)[:5]}{'...' if len(supplementaires) > 5 else ''}")
    
    # 3. Analyse des coefficients
    print(f"\nANALYSE DES COEFFICIENTS:")
    print(f"Termes linéaires: {len(bqm.linear)}")
    print(f"Termes quadratiques: {len(bqm.quadratic)}")
    
    coeffs_lineaires = list(bqm.linear.values())
    coeffs_quadratiques = list(bqm.quadratic.values())
    
    if coeffs_lineaires:
        print(f"Plage coefficients linéaires: [{min(coeffs_lineaires):.2f}, {max(coeffs_lineaires):.2f}]")
    if coeffs_quadratiques:
        print(f"Plage coefficients quadratiques: [{min(coeffs_quadratiques):.2f}, {max(coeffs_quadratiques):.2f}]")
    
    
    

# Les paramètres du prb
B = [0, 1]  # Noeuds
L = [(0, 1)]  # Lignes (non orientés)
c_i = {0: 10, 1: 5}          # Coûts par nœud
D_i = {0: 3, 1: 3}           # Demande par noeud
B_ij = {(0,1): 1}              # susceptance
lambda_val = 10000          # Coefficient de pénalité

# Bornes maximales 
P_i_max = {0: 4, 1: 6}     # Valeurs maximales pour P_i
L_ij_max = {(0,1): 3}           # Valeurs maximales pour L_ij

P_i = {0: 0, 1: 6}           # Valeurs cibles P_i
L_ij = {(0,1): -3}               # flux entre deux noeuds

# Calcul du nombre de bits nécessaires
p_i = {i: math.ceil(math.log2(P_i_max[i] + 1)) for i in B}
k_ij = {(i,j): math.ceil(math.log2(L_ij_max[(i,j)] + 1)) for (i,j) in L}
K_ij = {(i,j): B_ij[(i,j)] * np.pi / 180.0 for (i,j) in L}  # Facteur de conversion

# CONSTRUCTION DU QUBO
bqm = dimod.BinaryQuadraticModel.empty(dimod.BINARY)

# Dictionnaires pour suivre toutes les variables créées
variables_info = {
    'beta_vars': {},
    'tilde_c_vars': {},
    'c1_vars': {},
    'c2_vars': {}
}

# Ajout des variables binaires
# Variables β (angles)
beta_vars = {}
for i in B:
    for m in range(9):
        beta_vars[(i, m)] = f'beta_{i}_{m}'
        bqm.add_variable(beta_vars[(i, m)], 0.0)
        variables_info['beta_vars'][(i, m)] = beta_vars[(i, m)]

# Variables de correction pour P_i
tilde_c_vars = {}
for i in B:
    for r in range(int(p_i[i])):
        tilde_c_vars[(i, r)] = f'tilde_c_{i}_{r}'
        bqm.add_variable(tilde_c_vars[(i, r)], 0.0)
        variables_info['tilde_c_vars'][(i, r)] = tilde_c_vars[(i, r)]

# Variables de correction pour L_ij (deux sets par lien)
c1_vars = {}
c2_vars = {}
for (i,j) in L:
    for p in range(int(k_ij[(i,j)])):
        c1_vars[(i,j,p)] = f'c1_{i}_{j}_{p}'
        c2_vars[(i,j,p)] = f'c2_{i}_{j}_{p}'
        bqm.add_variable(c1_vars[(i,j,p)], 0.0)
        bqm.add_variable(c2_vars[(i,j,p)], 0.0)
        variables_info['c1_vars'][(i,j,p)] = c1_vars[(i,j,p)]
        variables_info['c2_vars'][(i,j,p)] = c2_vars[(i,j,p)]

# Terme de coût principal (linéaire)
for i in B:
    for (i,j) in L:
        if (i,j) in K_ij:
            for m in range(8):
                coeff = c_i[i] * K_ij[(i,j)] * (2**m)
                
                # Terme β^i_m
                bqm.add_linear(beta_vars[(i, m)], coeff)
                
                # Terme β^j_m
                bqm.add_linear(beta_vars[(j, m)], -coeff)

# Contraintes quadratiques (pénalités)
def add_quadratic_penalty(bqm, expression, lambda_val, target):
    """
    Ajoute une pénalité quadratique λ*(expression - target)^2 au BQM.
    """
    # Développement de (expression - target)^2
    for (var1, coeff1) in expression:
        # Terme linéaire: coeff1^2 * var1
        bqm.add_linear(var1, lambda_val * coeff1**2)
        
        # Terme croisé: 2*coeff1*coeff2 * var1*var2
        for (var2, coeff2) in expression:
            if var1 < var2:  # Évite les doublons
                bqm.add_quadratic(var1, var2, 2 * lambda_val * coeff1 * coeff2)
    
    # Terme -2*λ*target*expression
    for (var, coeff) in expression:
        bqm.add_linear(var, -2 * lambda_val * target * coeff)

# Contraintes pour P_i (nœuds)
for i in B:
    expr = []
    
    # Partie angulaire
    for (i,j) in L:
        if (i,j) in K_ij:
            for m in range(8):
                expr.append((beta_vars[(i, m)], K_ij[(i,j)] * (2**m)))
                expr.append((beta_vars[(j, m)], -K_ij[(i,j)] * (2**m)))
    
    # Variables de correction
    for r in range(int(p_i[i])):
        expr.append((tilde_c_vars[(i, r)], 2**r))
    
    # Ajout de la constante D_i et de la cible P_i
    add_quadratic_penalty(bqm, expr, lambda_val, P_i[i] - D_i[i])

# Contraintes pour L_ij (liens)
for (i,j) in L:
    if (i,j) in K_ij:
        # Première contrainte (c^2_{p,i,j})
        expr1 = []
        for m in range(8):
            expr1.append((beta_vars[(j, m)], K_ij[(i,j)] * (2**m)))
            expr1.append((beta_vars[(i, m)], -K_ij[(i,j)] * (2**m)))
        for p in range(int(k_ij[(i,j)])):
            expr1.append((c2_vars[(i,j,p)], 2**p))
        add_quadratic_penalty(bqm, expr1, lambda_val, L_ij[(i,j)])
        
        # Deuxième contrainte (c^1_{p,i,j})
        expr2 = []
        for m in range(8):
            expr2.append((beta_vars[(i, m)], K_ij[(i,j)] * (2**m)))
            expr2.append((beta_vars[(j, m)], -K_ij[(i,j)] * (2**m)))
        for p in range(int(k_ij[(i,j)])):
            expr2.append((c1_vars[(i,j,p)], 2**p))
        add_quadratic_penalty(bqm, expr2, lambda_val, L_ij[(i,j)])

print(verifier_construction_bqm(bqm, variables_info))


VÉRIFICATION DE LA CONSTRUCTION DU BQM
✓ Matrice symétrique: True
✓ Variables manquantes: 0
✓ Variables supplémentaires: 0

ANALYSE DES COEFFICIENTS:
Termes linéaires: 28
Termes quadratiques: 237
Plage coefficients linéaires: [-84155.11, 800000.00]
Plage coefficients quadratiques: [-399268.14, 357443.43]
None


# 3 noeuds

## Méthode classique

In [5]:
from gurobipy import Model, GRB


nodes = [0, 1,2]
P_min = {0: 0, 1: 0, 2:0}
P_max = {0: 5, 1: 10, 2: 0}
c = {0: 15, 1: 5, 2:0}
d = {0: 2, 1: 3, 2:4}

lines = [(0,1), (1,2), (0,2)]
L_max = {(0,1): 6, (1,2): 6, (0,2): 4}
B = {(0,1): 1.0, (1,2): 1, (0,2):1}  # susceptance
X = {(0,1): 1.0 / B[(0,1)], (1,2): 1.0 / B[(1,2)], (0,2): 1.0 / B[(0,2)] }  # reactance



# Modèle
m = Model("OTS")
m.setParam('OutputFlag', 0)

# Variables
g = m.addVars(nodes, lb=0, ub=GRB.INFINITY, name="g")
l = m.addVars(lines, lb=-GRB.INFINITY, ub=GRB.INFINITY, name="l")
theta = m.addVars(nodes, lb=-GRB.INFINITY, ub=GRB.INFINITY, name="theta")
# delta = m.addVars(lines, vtype=GRB.BINARY, name="delta")  # Supprimé pour OPF



# Contraintes

for i in nodes:
    m.addConstr(g[i] >= P_min[i], name=f"Pmin_{i}")
    m.addConstr(g[i] <= P_max[i], name=f"Pmax_{i}")

m.addConstr(theta[0] == 0, name="ref_angle")

for i in nodes:
    flux_sum = 0
    for (u,v) in lines:
        if u == i:
            flux_sum += l[(u,v)]
        elif v == i:
            flux_sum -= l[(u,v)]
    m.addConstr(g[i] - d[i] == flux_sum, name=f"balance_{i}")

# Suppression des contraintes sur delta et modification des contraintes sur l:

for (i,j) in lines:
    # On considère la ligne toujours fermée => delta=1 implicite
    m.addConstr(l[(i,j)] <= L_max[(i,j)], name=f"cap_pos_{i}_{j}")
    m.addConstr(l[(i,j)] >= -L_max[(i,j)], name=f"cap_neg_{i}_{j}")

    # Contraintes DC sans delta ni Big M
    m.addConstr(l[(i,j)] == (theta[i] - theta[j]) / X[(i,j)], name=f"dc_{i}_{j}")

# Objectif
m.setObjective(sum(c[i]*g[i] for i in nodes), GRB.MINIMIZE)

# Résolution
m.optimize()

# Affichage
if m.status == GRB.OPTIMAL:
    print("Solution optimale trouvée:")
    for i in nodes:
        print(f"g[{i}] = {g[i].X:.2f}")
    for (i,j) in lines:
        print(f"l[{i},{j}] = {l[(i,j)].X:.2f}")
        print(f"theta[{i}] = {theta[i].X:.2f}")
else:
    print("Pas de solution optimale trouvée.")
    

print(theta)


Restricted license - for non-production use only - expires 2026-11-23
Solution optimale trouvée:
g[0] = 0.00
g[1] = 9.00
g[2] = 0.00
l[0,1] = -2.67
theta[0] = 0.00
l[1,2] = 3.33
theta[1] = 2.67
l[0,2] = 0.67
theta[0] = 0.00
{0: <gurobi.Var theta[0] (value 0.0)>, 1: <gurobi.Var theta[1] (value 2.666666666666667)>, 2: <gurobi.Var theta[2] (value -0.6666666666666667)>}


## Méthode quantique: QA

In [ ]:
import numpy as np
from dwave.samplers import SimulatedAnnealingSampler
import itertools

# Paramètres du problème
B = [0, 1, 2]  # Noeuds
L = [(0, 1), (0, 2), (1, 2)]  # Lignes
c_i = {0: 15, 1: 5, 2: 0}  # Coûts par nœud
D_i = {0: 2, 1: 3, 2: 4}  # Demande par nœud
Pi=[0,9,0] #la cible---> pour faire la comparaison après
P_i_max = {0: 5, 1: 10, 2: 0}  # Capacité de production max
L_ij_max = {(0, 1): 6, (0, 2): 4, (1, 2): 6}  # Capacité des lignes
lambda_ = 500 # Paramètre de pénalité

# Calcul des bits nécessaires
def bits_required(max_val):
    return int(np.ceil(np.log2(max_val + 1))) if max_val > 0 else 0

p_i = {i: bits_required(P_i_max[i]) for i in B}  # Bits pour la production
k_ij = {edge: bits_required(L_ij_max[edge]) for edge in L}  # Bits pour les capacités

variables = {}
index_map = {}
counter = 0

# Variables bêta (angles)
for i in B:
    for m in range(9):
        var_name = f'beta_{i}_{m}'
        variables[var_name] = counter
        index_map[counter] = ('beta', i, m)
        counter += 1

# Variables de production
for i in B:
    if p_i[i] > 0:
        for r in range(p_i[i]):
            var_name = f'prod_{i}_{r}'
            variables[var_name] = counter
            index_map[counter] = ('prod', i, r)
            counter += 1

# Variables de capacité (c1 et c2)
for (i, j) in L:
    for p in range(k_ij[(i, j)]):
        var_name = f'c1_{i}_{j}_{p}'
        variables[var_name] = counter
        index_map[counter] = ('c1', i, j, p)
        counter += 1
        
    for p in range(k_ij[(i, j)]):
        var_name = f'c2_{i}_{j}_{p}'
        variables[var_name] = counter
        index_map[counter] = ('c2', i, j, p)
        counter += 1

n_vars = counter
Q = np.zeros((n_vars, n_vars))

# Coût de production
for i in B:
    for (u, v) in L:
        if u == i or v == i:
            if u == i:
                j = v
            else:
                j = u
            for m in range(9):
                var_i = f'beta_{i}_{m}'
                var_j = f'beta_{j}_{m}'
                idx_i = variables[var_i]
                idx_j = variables[var_j]
                # Coefficient pour beta_i^m
                Q[idx_i, idx_i] += c_i[i] * (np.pi/180) * (2**m)
                # Coefficient pour beta_j^m
                Q[idx_j, idx_j] -= c_i[i] * (np.pi/180) * (2**m)

#Contrainte de production
for i in B:
    expr_terms = []
    const = D_i[i] - P_i[i]
    
    # Flux incidents
    for (u, v) in L:
        if u == i or v == i:
            if u == i:
                j = v
            else:
                j = u
            for m in range(9):
                var_i = f'beta_{i}_{m}'
                var_j = f'beta_{j}_{m}'
                expr_terms.append((var_i, (np.pi/180) * (2**m)))
                expr_terms.append((var_j, -(np.pi/180) * (2**m)))
    
    # Production
    if p_i[i] > 0:
        for r in range(p_i[i]):
            var_prod = f'prod_{i}_{r}'
            expr_terms.append((var_prod, 2**r))
    
    # Ajout des termes au QUBO
    n_terms = len(expr_terms)
    for k in range(n_terms):
        var1, coeff1 = expr_terms[k]
        idx1 = variables[var1]
        # Terme linéaire
        Q[idx1, idx1] += lambda_ * 2 * const * coeff1
        
        for l in range(k, n_terms):
            var2, coeff2 = expr_terms[l]
            idx2 = variables[var2]
            if idx1 == idx2:
                # Terme quadratique (carré)
                Q[idx1, idx1] += lambda_ * coeff1 * coeff2
            else:
                i_min, i_max = min(idx1, idx2), max(idx1, idx2)
                Q[i_min, i_max] += lambda_ * 2 * coeff1 * coeff2

# Contraintes de capacité
for (i, j) in L:
    # Sens i->j
    expr_terms1 = []
    const1 = -L_ij_max[(i, j)]
    for m in range(9):
        var_i = f'beta_{i}_{m}'
        var_j = f'beta_{j}_{m}'
        expr_terms1.append((var_i, (np.pi/180) * (2**m)))
        expr_terms1.append((var_j, -(np.pi/180) * (2**m)))
    
    for p in range(k_ij[(i, j)]):
        var_c1 = f'c1_{i}_{j}_{p}'
        expr_terms1.append((var_c1, 2**p))
    
    n_terms1 = len(expr_terms1)
    for k in range(n_terms1):
        var1, coeff1 = expr_terms1[k]
        idx1 = variables[var1]
        Q[idx1, idx1] += lambda_ * 2 * const1 * coeff1
        
        for l in range(k, n_terms1):
            var2, coeff2 = expr_terms1[l]
            idx2 = variables[var2]
            if idx1 == idx2:
                Q[idx1, idx1] += lambda_ * coeff1 * coeff2
            else:
                i_min, i_max = min(idx1, idx2), max(idx1, idx2)
                Q[i_min, i_max] += lambda_ * 2 * coeff1 * coeff2
    
    # Sens j->i
    expr_terms2 = []
    const2 = -L_ij_max[(i, j)]
    for m in range(9):
        var_i = f'beta_{i}_{m}'
        var_j = f'beta_{j}_{m}'
        expr_terms2.append((var_i, -(np.pi/180) * (2**m)))
        expr_terms2.append((var_j, (np.pi/180) * (2**m)))
    
    for p in range(k_ij[(i, j)]):
        var_c2 = f'c2_{i}_{j}_{p}'
        expr_terms2.append((var_c2, 2**p))
    
    n_terms2 = len(expr_terms2)
    for k in range(n_terms2):
        var1, coeff1 = expr_terms2[k]
        idx1 = variables[var1]
        Q[idx1, idx1] += lambda_ * 2 * const2 * coeff1
        
        for l in range(k, n_terms2):
            var2, coeff2 = expr_terms2[l]
            idx2 = variables[var2]
            if idx1 == idx2:
                Q[idx1, idx1] += lambda_ * coeff1 * coeff2
            else:
                i_min, i_max = min(idx1, idx2), max(idx1, idx2)
                Q[i_min, i_max] += lambda_ * 2 * coeff1 * coeff2

# Conversion en format QUBO plat
qubo_dict = {}
for i in range(n_vars):
    for j in range(i, n_vars):
        if Q[i, j] != 0:
            qubo_dict[(i, j)] = Q[i, j]

# Résolution avec Simulated Annealing
sampler = SimulatedAnnealingSampler()
response = sampler.sample_qubo(qubo_dict, num_reads=100)
sample = response.first.sample

# Décodage de la solution
solution = {var: sample[idx] for var, idx in variables.items()}

# Calcul des valeurs finales
def compute_angle(i):
    return sum(int(solution[f'beta_{i}_{m}']) * (2**m) for m in range(9))

# Corrige B_ij pour inclure les deux sens
B_ij = {}
for (i, j) in L:
    B_ij[(i, j)] = 1.0
    B_ij[(j, i)] = 1.0

def compute_flow(i, j):
    angle_diff = compute_angle(i) - compute_angle(j)
    return B_ij[(i, j)] * (np.pi/180) * angle_diff


def compute_production(i):
    if p_i[i] == 0:
        return 0
    return sum(int(solution[f'prod_{i}_{r}']) * (2**r) for r in range(p_i[i]))


# Affichage des résultats
print("\n" + "="*50)
print("Résultats optimaux:")
print("="*50)

# Angles
print("\nAngles (degrés):")
for i in B:
    angle_val = compute_angle(i)
    print(f"  Noeud {i}: {angle_val:.2f}°")

# Flux
print("\nFlux sur les lignes:")
for (i, j) in L:
    flow_ij = compute_flow(i, j)
    print(f"  Ligne ({i},{j}): {flow_ij:.2f} MW (Capacité: {L_ij_max[(i,j)]} MW)")

# Production
print("\nProduction:")
for i in B:
    prod_val = compute_production(i)
    target = P_i[i]
    print(f"  Noeud {i}: Production = {prod_val}")

# Détection des violations
print("\nVérification des contraintes:")
for i in B:
    net_flow = sum(compute_flow(i, j) for j in [v for u,v in L if u==i] + [u for u,v in L if v==i])
    balance = net_flow + D_i[i] + compute_production(i) - P_i[i]
    print(f"  Équilibre noeud {i}: {balance:.2f} (cible 0)")

for (i,j) in L:
    flow = compute_flow(i, j)
    cap1 = sum(solution[f'c1_{i}_{j}_{p}'] * (2**p) for p in range(k_ij[(i,j)]))
    cap2 = sum(solution[f'c2_{i}_{j}_{p}'] * (2**p) for p in range(k_ij[(i,j)]))
    print(f"  Capacité ligne ({i},{j}): Flux={flow:.2f}, c1={cap1}, c2={cap2}")

print("\n" + "="*50)
print(f"Valeur objective: {response.first.energy:.2f}")
print("="*50)



Résultats optimaux:

Angles (degrés):
  Noeud 0: 448.00°
  Noeud 1: 511.00°
  Noeud 2: 400.00°

Flux sur les lignes:
  Ligne (0,1): -1.10 MW (Capacité: 6 MW)
  Ligne (0,2): 0.84 MW (Capacité: 4 MW)
  Ligne (1,2): 1.94 MW (Capacité: 6 MW)

Production:
  Noeud 0: Production = 0
  Noeud 1: Production = 9
  Noeud 2: Production = 0

Vérification des contraintes:
  Équilibre noeud 0: 1.74 (cible 0)
  Équilibre noeud 1: 0.04 (cible 0)
  Équilibre noeud 2: 1.22 (cible 0)
  Capacité ligne (0,1): Flux=-1.10, c1=7, c2=5
  Capacité ligne (0,2): Flux=0.84, c1=3, c2=5
  Capacité ligne (1,2): Flux=1.94, c1=4, c2=7

Valeur objective: -206175.68
